# Simple neural network for XOR problem

In [1]:
import numpy as np
from tqdm import tqdm

In [2]:
# initialize
input_n = 2
output_n = 1
number_hidden = 1
h1_n = 2
h1_w = np.random.random((input_n+1,h1_n)) # from input to hidden layer weights
out_w = np.random.random((h1_n+1,output_n)) # from hidden layer to output weights
learning_rate = 0.75

In [3]:
# sigmoid acivation function
def activation(x): 
    return 1/(1+np.exp(-x))

In [4]:
# forward propogation
def forwarder(x):
    x = np.append(x,1) # add bias
    h1 = np.dot(x,h1_w) 
    h1_active = activation(h1)
    h1_active = np.append(h1_active,1) # add bias
    out = np.dot(h1_active,out_w)
    out_active = activation(out)
    return list([h1_active,out_active])

In [5]:
# backward propogation
def backwarder(x,y):
    fwd = forwarder(x)   
    error = y-fwd[1]
    delta_out = error*fwd[1]*(1-fwd[1])
    out_dw = learning_rate*delta_out*fwd[0]
    
    h1_ub = fwd[0]
    delta_h1 = h1_ub*(1-h1_ub)*np.dot(out_w,delta_out)
    h1_dw = learning_rate*h1_w*delta_h1[:-1]*np.append(x,1)[:, np.newaxis]
    return list([out_dw,h1_dw])

In [6]:
def epoch(x,y):
    out_all_dw  = 0
    h1_all_dw = 0
    for indep,dep in zip(x,y):
        back = backwarder(indep,dep)
        out_all_dw += back[0][:, np.newaxis]
        h1_all_dw += back[1]
    return list([out_all_dw,h1_all_dw])

In [7]:
def result(x):
    res = []
    for example in x:
        res.append(forwarder(example)[1])
    return res

In [8]:
# XOR data points
x_train = np.array([[1,1],
                    [0,0],
                    [0,1],
                    [1,0]])
y_train = np.array([[1],
                    [1],
                    [0],
                    [0]])

In [9]:
# pre-training details
loss = 0
print 'Pre training'
for i in range(len(x_train)):
    loss +=(y_train[i]-forwarder(x_train[i])[1] )**2
    print x_train[i],'  Predict:',result(x_train)[i],'  Target:',y_train[i],'  Error:',y_train[i]-forwarder(x_train[i])[1]
print 'Loss: ', loss

# train for 50,000 epochs
for i in tqdm(range(50000)):
    if i%10000==0 and i!=0:
        loss = 0
        for j in range(len(x_train)):
            loss +=(y_train[j]-forwarder(x_train[j])[1] )**2
        print 'Loss: ', loss,' for '+str(i)+'-th epoch'
    e = epoch(x_train,y_train)
    out_w += e[0]
    h1_w += e[1]

    
# post_training details    
loss = 0
print 'Post training'
for i in range(len(x_train)):
    loss +=(y_train[i]-forwarder(x_train[i])[1] )**2
    print x_train[i],'  Predict:',result(x_train)[i],'  Target:',y_train[i],'  Error:',y_train[i]-forwarder(x_train[i])[1]
print 'Loss: ', loss

Pre training
[1 1]   Predict: [ 0.73778455]   Target: [1]   Error: [ 0.26221545]
[0 0]   Predict: [ 0.69552934]   Target: [1]   Error: [ 0.30447066]
[0 1]   Predict: [ 0.72392033]   Target: [0]   Error: [-0.72392033]
[1 0]   Predict: [ 0.71691368]   Target: [0]   Error: [-0.71691368]
Loss:  [ 1.19948519]


 19%|██████████████▎                                                            | 9566/50000 [00:01<00:07, 5542.78it/s]

Loss:  [ 0.02980608]  for 10000-th epoch


 40%|█████████████████████████████▍                                            | 19894/50000 [00:03<00:05, 5825.46it/s]

Loss:  [ 0.01260752]  for 20000-th epoch


...

Loss:  [ 0.00786363]  for 30000-th epoch


 79%|██████████████████████████████████████████████████████████▎               | 39389/50000 [00:06<00:01, 5866.36it/s]

Loss:  [ 0.00568122]  for 40000-th epoch


100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:08<00:00, 5769.67it/s]


Post training
[1 1]   Predict: [ 0.9558878]   Target: [1]   Error: [ 0.0441122]
[0 0]   Predict: [ 0.98349198]   Target: [1]   Error: [ 0.01650802]
[0 1]   Predict: [ 0.03329023]   Target: [0]   Error: [-0.03329023]
[1 0]   Predict: [ 0.03329023]   Target: [0]   Error: [-0.03329023]
Loss:  [ 0.00443488]
